In [1]:
import os
os.environ["nnUNet_raw"] = "/mnt/d/nnunet/raw"
os.environ["nnUNet_preprocessed"] = "/mnt/d/nnunet/preprocessed"
os.environ["nnUNet_results"] = "/mnt/d/nnunet/results"

from nnunetv2.paths import nnUNet_preprocessed, nnUNet_results, nnUNet_raw

print(nnUNet_preprocessed, nnUNet_results, nnUNet_raw)

/mnt/d/nnunet/preprocessed /mnt/d/nnunet/results /mnt/d/nnunet/raw


In [2]:
import os
import torch
import shutil
from nnunet_api import NnUnetApi
from tools.data_reformat import data_prepare
from tools.json_pickle_stuff import copy_plans_json

# =================================================================================
# 1. Configuration
# =================================================================================
# --- Define the IDs and paths for your datasets and models ---

# ABS path to the raw data directory: each subject has its own folder with BraTS namind convention
RAW_DATA_PATH = "/mnt/d/Datasets/mehdi_code_test/sample_data" #"PATH_TO_SAMPLE_RAW_DATA"

# The Dataset ID of the model you want to use for pre-training.
# This model's architecture and plans will be transferred to your new dataset.
PRETRAINED_DATASET_ID = 770

# The Dataset ID of your new dataset that you want to fine-tune on.
# Make sure you have already converted this dataset to the nnU-Net format.
FINETUNE_DATASET_ID = 666

# An identifier for the new plans that will be created for your fine-tuning dataset.
# It's good practice to give it a descriptive name.
FINETUNE_PLANS_ID = 'nnUNetPlans_finetune_from_brats'

# The full path to the pre-trained model's checkpoint file.
# This file contains the weights that will be used to initialize your new model.
PRETRAINED_CHECKPOINT_PATH = os.path.join(nnUNet_results, "Dataset770_BraTSGLIPreCropRegion/nnUNetTrainer__nnUNetResEncUNetPlans__3d_fullres/fold_0/checkpoint_final.pth")

# The GPU device to use for training.
# DEVICE = torch.device('cuda')

# Fold number you want to train (on the fine tunning dataset)
FOLD = 0

# NUMBER of epochs to train the model
N_EPOCHS = 3

# Initial Learning Rate for model training
INIT_LR = 1e-3

# ABS Path to the folder where testing data is located
INPUT_FOLDER_INFER = "/mnt/d/Datasets/mehdi_code_test/sample_test_decathlon" #'PATH_TO_TESTING_DATA'

# ABS Path to the folder where the results of inference will be saved
OUTPUT_FOLDER_INFER_FINETUNE = './test_ftune' #'PATH_TO_SAVE_RESULTS_FINETUNE'
OUTPUT_FOLDER_INFER_PRETRAINED = './test_pretrain' #'PATH_TO_SAVE_RESULTS_PRETRAINED'


In [3]:
# ## --- Step 00: Preparing the raw dataset into Decathlon format
print("\nStep 0: Running Data Preparation script...")
DST_DATA_NAME = "Dataset"+str(FINETUNE_DATASET_ID)+"_finetune_Decathlon"
data_prepare(RAW_DATA_PATH, os.path.join(nnUNet_raw, DST_DATA_NAME))
n_case=len(os.listdir(os.path.join(nnUNet_raw, DST_DATA_NAME,"labelsTr")))
copy_plans_json("./dataset.json", os.path.join(nnUNet_raw, DST_DATA_NAME), n_case)



Step 0: Running Data Preparation script...
--------
data reformat in process for case 1 out of 10 ...
data reformat in process for case 2 out of 10 ...
data reformat in process for case 3 out of 10 ...
data reformat in process for case 4 out of 10 ...
data reformat in process for case 5 out of 10 ...
data reformat in process for case 6 out of 10 ...
data reformat in process for case 7 out of 10 ...
data reformat in process for case 8 out of 10 ...
data reformat in process for case 9 out of 10 ...
data reformat in process for case 10 out of 10 ...
--------
All files were reformated, ready for segmentation!


In [4]:

# =================================================================================
# 2. Initialize the API
# =================================================================================
api = NnUnetApi()
## --- Step 1: Extract the Fingerprint for Your New Dataset ---
## This step analyzes the properties of your new dataset (image sizes, spacings, etc.)
## and creates a "fingerprint" file. This is a prerequisite for any planning.
## This wraps: nnUNetv2_extract_fingerprint CLI
print(f"Step 1: Extracting fingerprint for Dataset {FINETUNE_DATASET_ID}...")
api.extract_fingerprint(
    finetune_dataset_id=FINETUNE_DATASET_ID
)
print("-> Fingerprint extracted.\n")

Step 1: Extracting fingerprint for Dataset 666...
Dataset666_finetune_Decathlon
-> Fingerprint extracted.



In [5]:
# --- Step 2: Load and Apply the Pre-trained Model's Plans ---
# This is the key step for aligning architectures. It takes the plans
# (network topology, patch size, normalization, etc.) from the pre-trained
# model and applies them to your new dataset's fingerprint, creating a new
# plan file specifically for fine-tuning.
# This wraps: nnUNetv2_move_plans_between_datasets CLI
print(f"Step 2: Applying plans from Dataset {PRETRAINED_DATASET_ID} to Dataset {FINETUNE_DATASET_ID}...")
api.apply_pretrained_plans(
    pretrained_dataset_id=PRETRAINED_DATASET_ID,
    finetune_dataset_id=FINETUNE_DATASET_ID,
    finetune_plans_identifier=FINETUNE_PLANS_ID
)
print(f"-> Plans applied. New plans identifier is '{FINETUNE_PLANS_ID}'.\n")


Step 2: Applying plans from Dataset 770 to Dataset 666...
-> Plans applied. New plans identifier is 'nnUNetPlans_finetune_from_brats'.



In [7]:
# --- Step 3: Preprocess Your New Dataset ---
# Now that your fine-tuning dataset has a valid (and aligned) plan file,
# you can run the standard preprocessing pipeline on it. This will resample,
# crop, and normalize your images according to the new plan.
# This wraps: nnUNetv2_preprocess CLI
print(f"Step 3: Preprocessing Dataset {FINETUNE_DATASET_ID} with the new plans...")
api.preprocess_dataset(
    dataset_id=FINETUNE_DATASET_ID,
    plans_identifier=FINETUNE_PLANS_ID
)
print("-> Preprocessing complete.\n")

Step 3: Preprocessing Dataset 666 with the new plans...
Preprocessing dataset Dataset666_finetune_Decathlon
Configuration: 3d_fullres...


100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:44<00:00,  4.46s/it]


-> Preprocessing complete.



In [8]:
# --- Step 4: Run the Fine-Tuning Training ---
# Finally, you can start the training process. This function works just like the
# standard `train` method but includes the `pretrained_weights` argument.
# nnU-Net will load the weights from the specified checkpoint file before starting
# the training, effectively fine-tuning the model on your new data.
# This wraps: nnUNetv2_train -pretrained_weights CLI
print(f"Step 4: Starting fine-tuning on Dataset {FINETUNE_DATASET_ID}...")
api.finetune(
    finetune_dataset_id=FINETUNE_DATASET_ID,
    configuration='3d_fullres',
    fold=FOLD,
    pretrained_checkpoint_path=PRETRAINED_CHECKPOINT_PATH,
    plans_identifier=FINETUNE_PLANS_ID,
    num_epochs=N_EPOCHS,
    initial_lr=INIT_LR
)
print("\nFine-tuning process has been started!")



Step 4: Starting fine-tuning on Dataset 666...
Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2025-10-21 10:51:09.433731: Using torch.compile...
################### Loading pretrained weights from file  /mnt/d/nnunet/results/Dataset770_BraTSGLIPreCropRegion/nnUNetTrainer__nnUNetResEncUNetPlans__3d_fullres/fold_0/checkpoint_final.pth ###################
Below is the list of overlapping blocks in pretrained model and nnUNet architecture:
encoder.stem.convs.0.conv.weight shape torch.Size([32, 4, 3, 3, 3])
encoder.stem.convs.0.conv.bias shape torch.Size([32])
encoder.stem.convs.0.norm.weight shape torch.Size([32])
encod

/home/locolinux2/miniconda3/envs/m_quant_py310/lib/python3.10/site-packages/fft_conv_pytorch/fft_conv.py:139: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:306.)
  output = output[crop_slices].contiguous()
/home/locolinux2/miniconda3/envs/m_quant_py310/lib/python3.10/site-packages/fft_conv_pytorch/fft_conv.py:139: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/pyt

using pin_memory on device 0


/home/locolinux2/miniconda3/envs/m_quant_py310/lib/python3.10/site-packages/fft_conv_pytorch/fft_conv.py:139: UserWarning: Using a non-tuple sequence for multidimensional indexing is deprecated and will be changed in pytorch 2.9; use x[tuple(seq)] instead of x[seq]. In pytorch 2.9 this will be interpreted as tensor index, x[torch.tensor(seq)], which will result either in an error or a different result (Triggered internally at /pytorch/torch/csrc/autograd/python_variable_indexing.cpp:306.)
  output = output[crop_slices].contiguous()



This is the configuration used by this training:
Configuration name: 3d_fullres
 {'data_identifier': 'nnUNetPlans_finetune_from_brats_nnUNetPlans_3d_fullres', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 3, 'patch_size': [128, 128, 128], 'median_image_size_in_voxels': [140.0, 171.0, 137.0], 'spacing': [1.0, 1.0, 1.0], 'normalization_schemes': ['ZScoreNormalization', 'ZScoreNormalization', 'ZScoreNormalization', 'ZScoreNormalization'], 'use_mask_for_norm': [True, True, True, True], 'resampling_fn_data': 'resample_data_or_seg_to_shape', 'resampling_fn_seg': 'resample_data_or_seg_to_shape', 'resampling_fn_data_kwargs': {'is_seg': False, 'order': 3, 'order_z': 0, 'force_separate_z': None}, 'resampling_fn_seg_kwargs': {'is_seg': True, 'order': 1, 'order_z': 0, 'force_separate_z': None}, 'resampling_fn_probabilities': 'resample_data_or_seg_to_shape', 'resampling_fn_probabilities_kwargs': {'is_seg': False, 'order': 1, 'order_z': 0, 'force_separate_z': None}, 'architecture': {'ne

InductorError: RuntimeError: Failed to find C compiler. Please specify via CC environment variable or set triton.knobs.build.impl.

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"


In [ ]:

# --- Step 5: Run the inference of the newly fine tunned model ---
# This wraps:
print(f"Step 5: Infering fine-tuned model on a testing dataset")
api.predict(
    input_folder=INPUT_FOLDER_INFER,
    output_folder=OUTPUT_FOLDER_INFER_FINETUNE,
    dataset_name_or_id=DST_DATA_NAME,
    plans_identifier=FINETUNE_PLANS_ID,
    configuration='3d_fullres',
    folds=[FOLD]
)



In [ ]:

## --- Step 5+: Run the inference of the original pretrained model ---
## This wraps:
print(f"Step 5: Infering pretrained model on a testing dataset")
api.predict(
    input_folder=INPUT_FOLDER_INFER,
    output_folder=OUTPUT_FOLDER_INFER_PRETRAINED,
    dataset_name_or_id="Dataset770_BraTSGLIPreCropRegion",
    plans_identifier="nnUNetResEncUNetPlans",
    configuration='3d_fullres',
    folds=[FOLD]
)